In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

e:\My Work\My Projects\FineTune_LLM\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

# pip install streamlit PyPDF2 transformers langchain

# chavinlo/alpaca-13b
# chavinlo/gpt4-x-alpaca
# decapoda-research/llama-7b-hf

# - `lmsys/vicuna-7b-delta-v1.1`: A 7 billion parameter version of Vicuna that is trained with delta updates⁴.
# - `stability-ai/stablevicuna`: A version of Vicuna that is further fine-tuned and trained with reinforced learning from human feedback³.
# - `chavinlo/vicuna-native`: A replica of Vicuna that is natively fine-tuned without using LORA¹.

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_8bit=True,
    device_map='auto',
    cache_dir='/path/to/cache/'
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


KeyboardInterrupt: 

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What is the capital of England?"

print(llm_chain.run(question))

In [ ]:
question = "What are alpacas? and how are they different from llamas?"

print(llm_chain.run(question))

In [ ]:
### With Memory

In [ ]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [ ]:
# We are going to set the memory to go back 4 turns
window_memory = ConversationBufferWindowMemory(k=4)

In [ ]:
conversation = ConversationChain(
    llm=local_llm, 
    verbose=True, 
    memory=window_memory
)

In [ ]:
conversation.prompt.template

In [ ]:
conversation.prompt.template = '''The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 

Current conversation:
{history}
Human: {input}
AI:'''

In [ ]:
conversation.predict(input="Hi there! I am Tapan")